In [50]:
#Uses python3

import numpy as np
import pandas as pd 
import time
import matplotlib
import matplotlib.pyplot as plt
import sys

In [77]:
def gen_e0(n):
    """генерирует все возможные последовательности из 7 спинов"""
    n_b = str(bin(n))[2:]
    n_arr = [-1]*(7-len(n_b)) + [int(x) if x=='1' else -1 for x in n_b]
    return n_arr

def calc_e_dict(arr):
    """вычисляет энергию конкретной конфигурации"""
    summa = (
        arr[1]*arr[2] + arr[2]*arr[3] + arr[3]*arr[4] +
        arr[4]*arr[5] + arr[5]*arr[6] + arr[6]*arr[1]
    )
    return -arr[0]*summa

def gen_e_dict():
    """генерируем и заоплняем словарь всех энергий"""
    d = dict()
    for i in range(0,128):
        conf = gen_e0(i)
        d[str(conf)] = calc_e_dict(conf)
    return d

def gen_state():
    """generate random init. state with lenght L*L and q=[-1,1]"""
    return np.array([np.random.choice([-1,1]) for _ in range(L*L)]).reshape(L,L)

def get_conf(s,i,j):
    """формируем конфигурацию, по вышеописанному правилу таблицы энергий"""
    arr = ([s[i,j]]+[s[i-1,j]]+[s[i,j+1]]+[s[i+1,j+1]]+
               [s[i+1,j]]+[s[i,j-1]]+[s[i-1,j-1]])
    return arr

def step(state, edict, T):
    """perform 1 Metropolis step"""
    j,k = np.random.randint(-1,L-1), np.random.randint(-1,L-1)
    arr = get_conf(state,j,k)
    e_jk = edict[str(arr)]
    if e_jk > 0:
        state[j,k] *= -1
    elif np.random.uniform() <= np.exp(2*e_jk/T): 
        state[j,k] *= -1

def mc_step(state, edict, T):
    """perform L*L flips for 1 MC step"""
    for _ in range(L**2):
        step(state, edict, T)
        
        
def model(T,N_avg=10,N_mc=10,Relax=10):
    """Моделируем АТ"""
    E, M = [], []

    state = gen_state()
    edict = gen_e_dict()
    
    #relax $Relax times be4 AVG
    for __ in range(Relax):
            mc_step(state, edict, T)
    #AVG every $N_mc steps
    for _ in range(N_avg):
        for __ in range(N_mc):
            mc_step(state, edict, T)
        E += [calc_e(state)]
        M += [calc_m(state)]
    
    return E, M
        
#===========================================================================================

def create_mask(L):
    """маска в виде 3 под-решёток"""
    a = np.asarray([i % 3 for i in range(L)])
    return (a + a[:, None])%3

def calc_e(st):
    """calculate energy per site
        # expland state matrix"""
    a = np.concatenate((st[L-1].reshape(1,L), st, st[0].reshape(1,L)), axis=0) 
    b = np.concatenate((a[:,-1].reshape(L+2,1),a,a[:,0].reshape(L+2,1)), axis=1)
    return -np.sum(b[1:-1, 1:-1]*b[2:, 2:]*(b[2:, 1:-1]+b[1:-1, 2:]))/(L*L)  

def calc_ms(st):
    """magnetization"""
    msr = np.array([np.sum(st[mask==i]) for i in [0,1,2]])/(L*L)
    return np.sqrt(np.sum(msr*msr))



In [70]:
global L, mask
L = 100
mask = create_mask(L)

np.random.seed(1)
s = gen_state(); nei = gen_e_dict()

%timeit [mc_step(s, nei, 1) for _ in range(10)]

2.32 s ± 193 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
global L, mask
L = 100
mask = create_mask(L)

np.random.seed(1)
s = gen_state(); edict = gen_e_dict()
print(calc_e(s))

for _ in range(10):
    mc_step(s, edict, 1)
print(calc_e(s))

-0.0048
-1.1


In [61]:
# py     -- 2.16 s ± 52.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)     # original
